<a href="https://colab.research.google.com/github/RazerRaymond/CrunchBase_Predictor/blob/main/int_mp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Early stage startup model**
*This is an model for early start-up evaluation for interview purposes at Miracle Plus China*

Data used in model grabbed from GitHub user @notpeter from [Link](https://github.com/notpeter/crunchbase-data).

All relavent data are copyright reserved for Crunchbase. Do not redistribute or use for any business uses.


## Rationales
Because It's hard to compute ROI given that dataset has no IPO data (Crunchbase actually has that dataset, but it would require enterprise access), I decice to work with acquired firms only. Not only this gives a more accruate estimates, but also reduce the time to train the model. 

### Possible Flaws
首先sample是有些biased，因为我data cleaning用到的一些techique是drop null values， 大公司或者更好的startups更应该有完整的profile所以选到的有bias
其次companies的categories我只选了第一个，实际上可以用list 列出所有的再one hot encoding但是这样会比较麻烦，sample数也不太够
最后的regression结果可以根据公司需求的roi来选择是否投资，我觉得3-5的cutoff都很不错

### Data Cleaning
In order to run and fit the data, we have to make sure our data is numerical.
Csv doesn't automatically do that, and we also need to clean all the Null values of our interest

In [1]:
# libraries needed
import pandas as pd
import numpy as np
import datetime
import csv
from scipy.stats import skewnorm 

In [2]:
# import datas
#inv_df = pd.read_csv('https://raw.githubusercontent.com/notpeter/crunchbase-data/master/investments.csv')

# Data Cleaning - parse all time object to datetime type
date_parser = lambda c: pd.to_datetime(c, format='%Y-%m-%d', errors='coerce')
acq_df = pd.read_csv('https://raw.githubusercontent.com/notpeter/crunchbase-data/master/acquisitions.csv', parse_dates=['acquired_at'], date_parser=date_parser)
comp_df = pd.read_csv('https://raw.githubusercontent.com/notpeter/crunchbase-data/master/companies.csv', parse_dates=['first_funding_at', 'last_funding_at'], date_parser=date_parser)

In [3]:
# Data Cleaning
# To clean the data, we manually convert all time columns of interest that is null
acq_l = ['price_amount','acquired_at']
comp_l = ['category_list', 'status', 'country_code', 'funding_rounds', 'first_funding_at', 'last_funding_at']
acq_df = acq_df.dropna(subset=acq_l)
comp_df = comp_df.dropna(subset=comp_l)

comp_df = comp_df[comp_df.funding_total_usd.apply(lambda x: x.isnumeric())]

Merging two dataframes to get one that we really want and train later

In [4]:
# Merge two dataframe to get the results that we want on acquired firms
df = pd.merge(acq_df, comp_df, left_on='company_permalink', right_on='permalink')
inted_list = ['funding_rounds', 'funding_total_usd','category_list', 'country_code', 'status', 'first_funding_at', 'last_funding_at', 'acquired_at', 'price_amount']
df = df[inted_list]
df['funding_total_usd'] = df['funding_total_usd'] .astype(np.float64)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1491 entries, 0 to 1490
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   funding_rounds     1491 non-null   int64         
 1   funding_total_usd  1491 non-null   float64       
 2   category_list      1491 non-null   object        
 3   country_code       1491 non-null   object        
 4   status             1491 non-null   object        
 5   first_funding_at   1491 non-null   datetime64[ns]
 6   last_funding_at    1491 non-null   datetime64[ns]
 7   acquired_at        1491 non-null   datetime64[ns]
 8   price_amount       1491 non-null   float64       
dtypes: datetime64[ns](3), float64(2), int64(1), object(3)
memory usage: 116.5+ KB


### Data Enrichment
It's biased to run simply on acquired firms, since not all invested firms (I guess only very small percent) are acquired. I decided to enrich the data set with 10000 entries, with their value now being transformed by an left-skewed normal distribution from their total investment to reflect their values for now.

*This part has certain flaw since ROI are surely not independent of investment amount, and will draw more problems when making investment decisions. I just assumed ROI as a single consideration for the sake of simplicity.*

In [5]:
# select first 10000 rows and select interest value
added = comp_df.head(10000)
added = added[['funding_rounds', 'funding_total_usd','category_list', 'country_code', 'status', 'first_funding_at', 'last_funding_at']]

#manually set date to 2016-01-01 and do cleaning
added['acquired_at'] = datetime.datetime(2016, 1, 1, 0, 0)
added['funding_total_usd'] = added['funding_total_usd'] .astype(np.float64)

# transform using skewed normal dist
added['price_amount'] = added.apply(lambda x : skewnorm.rvs(a=5, loc = 0.1, scale =0.25)*x.funding_total_usd, axis = 1)

#update our data frame
df = pd.concat([df,added]).drop_duplicates().reset_index(drop=True)

In [6]:
# kept the first category(flawed)
# convert some of the key info into form of data
# convert to time dif instead of time
df['category'] = df['category_list'].str.split('|').str[0]
df['invest_period'] = df['last_funding_at'] - df['first_funding_at'] 
df['invest_period'] = df['invest_period']/np.timedelta64(1,'M')
df['payout period'] = df['acquired_at'] - df['last_funding_at'] 
df['payout period'] = df['payout period']/np.timedelta64(1,'M')
# ROI instead of amount 
df['ROI'] = ((df['price_amount'] - df['funding_total_usd']) / df['funding_total_usd'])


In [7]:
# Clean the final data set
# hot_cc = pd.get_dummies(df['country_code'])
# hot_st = pd.get_dummies(df['status'])
df = df.drop(columns = ['funding_total_usd', 'acquired_at', 'first_funding_at', 'last_funding_at', 'category_list'])
# df = df.join(hot_cc)
# df = df.join(hot_st)
#df = df['funding_rounds','funding_total_usd','category_list','country_code','status', 'invest_period']

In [8]:
# Rearrange final df's columns

#list(df.columns.values)
rear = ['category',
 'country_code',
 'status',
 'funding_rounds',
 'price_amount',
 'invest_period',
 'payout period',
 'ROI']
df = df[rear]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11489 entries, 0 to 11488
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   category        11489 non-null  object 
 1   country_code    11489 non-null  object 
 2   status          11489 non-null  object 
 3   funding_rounds  11489 non-null  int64  
 4   price_amount    11489 non-null  float64
 5   invest_period   11489 non-null  float64
 6   payout period   11489 non-null  float64
 7   ROI             11489 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 718.2+ KB


### Training
Now we try to fit a neural network to predict ROI

In [10]:
#import libs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

# Encoding the data
df = pd.get_dummies(df)

train, test = train_test_split(df, test_size=0.1)


y_train = train['ROI'].values
y_test = test['ROI'].values
train.drop('ROI', axis=1)
test.drop('ROI', axis=1)
x_ta = train.values
x_te = test.values

# normalize data
scaler = StandardScaler()
X_train = scaler.fit_transform(x_ta)
X_test = scaler.fit_transform(x_te)

# testing alpha alpha = 0.0003
# regr_cv = RidgeCV(alphas=[0.0003, 0.0001, 0.0005])
# model_cv = regr_cv.fit(X_train, y_train)
# model_cv.alpha_
ridge = linear_model.Ridge(alpha=0.0003, normalize=False)
ridge.fit(X_train, y_train)
pred_train_rr= ridge.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_train_rr)))
print(r2_score(y_test, pred_train_rr))



56.055178726179086
0.9401277893629795


*The interesting train test split is inspired by [here](https://stackoverflow.com/questions/24147278/how-do-i-create-test-and-train-samples-from-one-dataframe-with-pandas), as well as one hot encoding [here](https://stackoverflow.com/questions/34007308/linear-regression-analysis-with-string-categorical-features-variables)
and standardization CV picking from [here](https://chrisalbon.com/machine_learning/linear_regression/selecting_best_alpha_value_in_ridge_regression/)*
